# Decision Tree (CART) from scratch

Below you can find the implementation from strach of a Decision Tree Algorithm using CART. 

The final model achieves a 98% accuracy when applied on the Iris Dataset.

For more details regarding the inspiration for this code, don't forget to check out [Sujan Dutta](https://github.com/Suji04/ML_from_Scratch/blob/master/decision%20tree%20classification.ipynb) work and also his (video)[(https://www.youtube.com/watch?v=sgQAhG5Q7iY&ab_channel=NormalizedNerd)] on the topic

#### Imports

In [ ]:
from sklearn.datasets import load_iris
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from toolz import curry
from statistics import mode

#### Loading the data and splitting into train and test

In [2]:
iris = load_iris()

In [ ]:
X = pd.DataFrame(iris.data, columns = iris.feature_names)
y = pd.DataFrame(iris.target, columns = ['type'])

#### Node Class implementation

This node class will be the basis for saving the decision tree as we build it. The ideia is to instanciate a Node and for each following iteration, save the respective branches into the Node itself

In [ ]:
class Node():
    def __init__(self, feature_name = None, threshold = None, left_subtree = None, right_subtree = None, information_gain = None, value = None):
        self.feature_name = feature_name
        self.threshold = threshold
        self.left_subtree = left_subtree
        self.right_subtree = right_subtree
        self.information_gain = information_gain
        self.value = value

#### Decision Tree Class

Below you can find the model class, which contains functions such as: 

- `calculate_impurity` to evaluate how pure the splits are
- `get_best_split` to find iteractively the best feature and threshold that maximizes the information gain
- `fit` which is a recursive function that evaluates the best split and saves the subtrees into a Node class
- `predict` that given a sample, make the way through the tree and return the majority class of the leaf node

In [372]:
class DecisionTreeClassifier():

    def __init__(self, max_depth=2, min_num_samples = 2):
        self.max_depth = max_depth
        self.min_num_samples = min_num_samples
        
    def calculate_impurity(self, data: pd.DataFrame, method: str = "gini"):
            assert method in ["gini", "entropy"], "Not a valid method"
            if method == "entropy":
                impurity = 0
                classes_prob = data.value_counts(normalize=True)
                for class_name in classes_prob.keys():
                    impurity -= classes_prob[class_name] * np.log2(classes_prob[class_name])
                return impurity
            impurity = 0
            classes_prob = data.value_counts(normalize=True)
            for class_name in classes_prob.keys():
                impurity += classes_prob[class_name] * (1 - classes_prob[class_name])
            return impurity
        
    def get_best_split(self, data: pd.DataFrame):        
        best_split = {}

        X = data.iloc[:,:-1]
        y = data.iloc[:,-1]

        # The negative infity ensures the first iteration will be valid, even in case of a pure node (information gain==0)
        best_information_gain = np.NINF

        for feature in X.columns:

            for threshold in X[feature].unique():

                #Split the dataset into left and right according to the threshold
                right_df, right_target = X[X[feature] >= threshold], y[X[feature] >= threshold]
                left_df, left_target = X[X[feature] < threshold], y[X[feature] < threshold]

                #Calculate the impurity of the split
                impurity_method = "gini"
                right_imp = self.calculate_impurity(right_target, impurity_method)
                left_imp = self.calculate_impurity(left_target, impurity_method)
                father_node_imp = self.calculate_impurity(y, impurity_method)

                #Calculate the information gain
                weights = [right_df.shape[0], left_df.shape[0]]
                impurity_weighted = np.average([right_imp, left_imp], weights = weights)
                information_gain = father_node_imp - impurity_weighted

                #Save the current best split
                if information_gain > best_information_gain:
                    best_split['feature'] = feature
                    best_split['threshold'] = threshold
                    best_split['right_df'] = pd.concat([right_df, right_target], axis = 1)
                    best_split['left_df'] = pd.concat([left_df, left_target], axis = 1)
                    best_split['information_gain'] = information_gain
                    best_information_gain = information_gain

        return best_split
    
    def fit(self, data, current_depth = 0):
    
        if current_depth <= max_depth and data.shape[0]>=min_samples: # Check stopping conditions
            best_split = self.get_best_split(data)

            if best_split['information_gain']>0: #This if statement will only occur if the node isn't pure
                right_tree = self.fit(best_split['right_df'], current_depth + 1)
                left_tree = self.fit(best_split['left_df'], current_depth + 1)

                return Node(best_split['feature'], best_split['threshold'], left_tree, right_tree, best_split['information_gain'])

        value = mode(data['type'])  #If the node is pure, we can return the majority class
        return Node(value = value)
    
    @curry
    def predict(self, sample, tree):    
        if tree.value == None: #If the node isn't a leaf node, we can make our way through the tree until we reach one
            if sample[tree.feature_name] >= tree.threshold:
                return self.predict(sample, tree.right_subtree)
            if sample[tree.feature_name] < tree.threshold:
                return self.predict(sample, tree.left_subtree)
        else:
            return tree.value #In case we end in a leaf node, return the associated value (majority class)

In [373]:
clf = DecisionTreeClassifier(max_depth = 2, min_num_samples= 2)

In [374]:
X_train, X_test, y_train, y_test  = train_test_split(X, y, test_size = 0.33, random_state = 42)

In [375]:
train_set = pd.concat([X_train, y_train], axis = 1)

In [376]:
trained_clf = clf.fit(train_set)

In [377]:
apply_pred = lambda x: clf.predict(x, trained_clf)

y_pred = X_test.apply(apply_pred, axis =1)

In [382]:
print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        19
           1       1.00      0.94      0.97        16
           2       0.94      1.00      0.97        15

    accuracy                           0.98        50
   macro avg       0.98      0.98      0.98        50
weighted avg       0.98      0.98      0.98        50

